In [26]:
import h5py
import random
import numpy as np
import tensorflow as tf
from datetime import datetime
import H5pyHelper
from tensorflow.keras import layers, models, regularizers, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import dask.array as da
from dask.array.slicing import shuffle_slice
import cv2
import pandas as pd
#from sklearn.metrics import accuracy_score

In [27]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
BATCH_SIZE = 32

Num GPUs Available:  0


In [28]:
image_datagen = ImageDataGenerator(validation_split=0.3,rescale= 1.0/255.0)
df = pd.read_csv("/workspaces/flora_dex/clean_data/image_paths.csv")

train_iterator = image_datagen.flow_from_dataframe(df,x_col="image-path",y_col="label",batch_size=32,class_mode="categorical",subset="training")
validate_iterator = image_datagen.flow_from_dataframe(df,x_col="image-path",y_col="label",batch_size=32,class_mode="categorical",subset="validation")

Found 23299 validated image filenames belonging to 102 classes.
Found 9985 validated image filenames belonging to 102 classes.


In [29]:
## alex net
model = models.Sequential([
    layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(500,500,3)),
    layers.BatchNormalization(),
    layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    layers.BatchNormalization(),
    layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    layers.BatchNormalization(),
    layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    layers.BatchNormalization(),
    layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    layers.BatchNormalization(),
    layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    layers.Flatten(),
    layers.Dense(4096, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(4096, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(NUM_CLASSES, activation='softmax')
])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 123, 123, 96)      34944     
_________________________________________________________________
batch_normalization_10 (Batc (None, 123, 123, 96)      384       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 61, 61, 96)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 61, 61, 256)       614656    
_________________________________________________________________
batch_normalization_11 (Batc (None, 61, 61, 256)       1024      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 30, 30, 256)       0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 30, 30, 384)      

In [30]:
#model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.optimizers.SGD(lr=0.001), metrics=['accuracy'])
model.compile(loss='categorical_crossentropy', optimizer=tf.optimizers.SGD(learning_rate=0.001), metrics=['accuracy'])

In [31]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)
fit = model.fit(x=train_iterator,
                steps_per_epoch= len(train_iterator),
                callbacks=[callback], 
                validation_steps= len(validate_iterator),
                validation_data=validate_iterator,
                verbose=1,
                epochs=250)

ImportError: Image transformations require SciPy. Install SciPy.

In [ ]:
def summarize_prediction(Y_true, Y_pred):
    #mse = mean_squared_error(Y_true, Y_pred)
    #accuracy = accuracy_score(Y_true, Y_pred)
    #print(f'mse       = {mse:.2}')
    print(f'accuracy = {accuracy:.2%}')
    
def predict_and_summarize(X, Y):
    Y_pred = model.predict(X)
    #summarize_prediction(Y, Y_pred)
    return Y_pred

hf = h5py.File('/home/sorozco0612/dev/flora_dex/raw_data/data.h5', 'r')
x_test = hf["x_test"]
y_test = hf["y_test"]

y_pred = predict_and_summarize(x_test, y_test)

print(y_pred[:1])
print(y_test[:1])

hf.close()